<a href="https://colab.research.google.com/github/sonomarina/Mathmodelling_DTU/blob/main/EX1_model2%2B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DATA FILES DIRECTORY
dirIn = 'path to Data folder for exercise 1'
# IMPORTS
import helpFunctions as hf
import matplotlib.pyplot as plt
import numpy as np
import imageio
# DAY ONE MULTISPECTRAL IMAGE AND ANNOTATIONS
multiIm, annotationIm = hf.loadMulti('multispectral_day01.mat','annotation_day01.png',\
dirIn)
# ASSUME EQUAL PROBABILITY. CHANGE THIS TO ACCOMMODATE METHOD 3 PRIOR PROBABILITIES
p = 0.5
# EXTRACT PIXELS FOR FAT AND MEAT
[fatPix, fatR, fatC] = hf.getPix(multiIm, annotationIm[:,:,1]);
[meatPix, meatR, meatC] = hf.getPix(multiIm, annotationIm[:,:,2]);
# MEANS FOR THE BANDS
meatM = np.mean(meatPix,0)
fatM = np.mean(fatPix, 0)
covM = np.zeros((19,19))
covF = np.zeros((19,19))
# ARRAY OF PIXELS FOR EACH BAND
m = 264196
reshIm = np.zeros((m, 19))
for i in range(0, 19):
reshIm[:,i] = multiIm[:,:,i].flatten()
# COMPUTE COVARIANCE
for i in range(0, 19):
for j in range(0, 19):
covM[i, j] = 1/(m - 1)*np.sum((reshIm[:,i] - meatM[i])*(reshIm[:,j] - meatM[j]))
covF[i, j] = 1/(m - 1)*np.sum((reshIm[:,i] - fatM[i])*(reshIm[:,j] - fatM[j]))
cov = 1/(2*m - 2)*((m-1)*covM + (m-1)*covF)

In [ ]:
# LDA FUNCTION

def discFunc(x):
  s_meat = np.dot(x,np.dot(np.linalg.inv(cov),meatM)) - \
1/2*np.dot(meatM, np.dot(np.linalg.inv(cov),meatM)) + np.log(p)
  s_fat = np.dot(x,np.dot(np.linalg.inv(cov),fatM)) - \
1/2*np.dot(fatM, np.dot(np.linalg.inv(cov),fatM)) + np.log(p)
  return s_meat, s_fat
# CLASSIFY THE PIXELS
s = np.zeros(264196)
for i in range(0, 264196):
  s_meat, s_fat = discFunc(reshIm[i,:])
  if (s_fat >= s_meat):
    s[i] = 1
  else:
      s[i] = 0
s = np.reshape(s, (514, 514)).astype(np.int64)
# IDENTIFIED CLASSIFICATIONS VS KNOWN CLASSIFICATIONS
c = s[fatR, fatC] == annotationIm[fatR, fatC, 1]
rate = len(c[c == True])/len(c)
# IDENTIFY BACKGROUND
bPix = np.where(np.logical_and(annotationIm[:,:,0] == 0, annotationIm[:,:,1] == 0, \
annotationIm[:,:,2] == 0))
# REMOVE BACKGROUND
s[bPix] = 0
# CLASSIFIED IMAGE PLOT
[fatPix, fatR, fatC] = hf.getPix(multiIm, s)
pixId = np.stack((fatR, fatC), axis=1)
imRGB = imageio.imread(dirIn + 'color_day01.png')
rgbOut = hf.setImagePix(imRGB, pixId)
plt.imshow(rgbOut)
plt.title('Classified image of the salami on day 1')
plt.xlim(0,514)
plt.ylim(514,0)
plt.savefig('salami_day1_model_2', dpi = 300)
plt.show()